In [2]:
from ultralytics import YOLO


model = YOLO("./weights/best.pt")

In [4]:
preds = model.track("../cv_training/detector_training/datasets/construction_analytics/videos/output118.mp4", stream=True)



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/359) /home/and/projects/hacks/construction_site_analytics/detector_inference/../cv_training/detector_training/datasets/construction_analytics/videos/output118.mp4: 384x640 3 diggers, 2 trucks, 3 cranes, 81.3ms
video 1/1 (2/359) /home/and/projects/hacks/construction_site_analytics/detector_inference/../cv_training/detector_training/datasets/construction_analytics/videos/output118.mp4: 384x640 3 diggers, 2 trucks, 3 cranes, 6.8ms
video 1/1 (3/3

In [6]:
boxes = preds[0].boxes

In [7]:
boxes

WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.yolo.engine.results.Boxes object with attributes:

boxes: tensor([[3.1059e+02, 7.9779e+02, 4.6170e+02, 9.1920e+02, 1.0000e+00, 9.5310e-01, 0.0000e+00],
        [2.3394e+03, 6.8754e+02, 2.4546e+03, 7.9869e+02, 2.0000e+00, 9.5179e-01, 3.0000e+00],
        [1.8657e+03, 3.7286e+02, 2.0965e+03, 8.1284e+02, 3.0000e+00, 9.4001e-01, 3.0000e+00],
        [2.2490e+03, 5.9061e+02, 2.3370e+03, 6.4760e+02, 4.0000e+00, 9.3045e-01, 0.0000e+00],
        [1.8874e+03, 6.0907e+02, 1.9985e+03, 6.5681e+02, 5.0000e+00, 9.1999e-01, 0.0000e+00],
        [2.3714e+03, 6.8429e+02, 2.4856e+03, 7.4026e+02, 6.0000e+00, 9.1093e-01, 3.0000e+00],
        [5.1194e+02, 5.8761e+02, 5.6208e+02, 6.2095e+02, 7.0000e+00, 7.5350e-01, 2.0000e+00],
        [4.6601e+02, 6.0229e+02, 5.2674e+02, 6.3301e+02, 8.0000e+00, 6.1705e-01, 2.0000e+00]])
cls: tensor([0., 3., 3., 0., 0., 3., 2., 2.])
conf: tensor([0.9531, 0.9518, 0.9400, 0.9305, 0.9200, 0.9109, 0.7535, 0.6171])
data: tensor([[3.1059e+02, 7.9779e+02, 4.6170e+02, 9

In [2]:
import cv2

cap = cv2.VideoCapture("../detector_training/datasets/construction_analytics/videos/output118.mp4")
_, frame = cap.read()

In [3]:
frame.shape

(1440, 2560, 3)

In [40]:
preds = model(frame, conf=0.6)[0].cpu()
boxes = preds.boxes
coords = boxes.xyxy.long()
classes = boxes.cls


0: 384x640 3 diggers, 2 trucks, 3 cranes, 7.9ms
Speed: 1.7ms preprocess, 7.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


In [41]:
coords

tensor([[ 310,  797,  461,  919],
        [2339,  687, 2454,  798],
        [1865,  372, 2096,  812],
        [2249,  590, 2336,  647],
        [1887,  609, 1998,  656],
        [2371,  684, 2485,  740],
        [ 511,  587,  562,  620],
        [ 466,  602,  526,  633]])

In [56]:
import numpy as np
import cv2
from ultralytics import YOLO


class Config: 
    weights = "./weights/best.pt"
    cls2name = {
        0: "digger",
        1: "tractor",
        2: "truck",
        3: "crane",
    }


class Detector: 
    def __init__(self, config, device="cpu") -> None:
        self.model = YOLO(config.weights)
        self.model.to(device)
        self.device = device
        self.cls2name = config.cls2name

    def detect(self, frame: np.ndarray, conf=0.6):
        preds = self.model(frame, conf=conf, verbose=False)[0].cpu().numpy()
        boxes = preds.boxes
        return boxes.xyxy.astype(int), boxes.cls, boxes.conf

    def process_video(self, video_src: str):
        results = []

        vidcap = cv2.VideoCapture(video_src)
        has_frame = True
        frame_num = 0
        while has_frame: 
            has_frame, frame = vidcap.read()

            if has_frame:
                results.append({"timestamp": f"frame:{frame_num}", "metadata": []})
                preds = self.detect(frame)
                
                for coords, cls, conf in zip(*preds):
                    results[-1]["metadata"].append(
                        {
                            "name": self.cls2name[cls],
                            "coords": coords.tolist(),
                            "conf": conf
                            "track_id": ...
                        }
                    )

            frame_num += 1

        vidcap.release()

        return results

In [57]:
detector = Detector(Config(), device="cuda")

In [58]:
preds = detector.process_video("../detector_training/datasets/construction_analytics/videos/output118.mp4")